In [1]:
import stock_data as stock

In [2]:
start = "2019-01-01"
end = "2022-12-31"
mode = "all"
query = "2330"

In [3]:
stock = stock.scrapy()
df = stock.get_spread_of_shareholding(start, end, mode, query)

 77%|███████▋  | 1365/1767 [38:20<10:44,  1.60s/it]

In [ ]:
df.to_csv("data/spread.csv", index = False)